## 使用Tabular Q-learning的方法实现一个小游戏。

例子的环境是一个一维世界, 在世界的右边有宝藏, 探索者只要得到宝藏尝到了甜头, 然后以后就记住了得到宝藏的方法, 这就是他用强化学习所学习到的行为。

Q-learning 是一种记录行为值 (Q value) 的方法, 每种在一定状态的行为都会有一个值 Q(s, a), 就是说 行为 a 在 s 状态的值是 Q(s, a)。s 在上面的探索者游戏中, 就是 o 所在的地点了。而每一个地点探索者都能做出两个行为 left/right, 这就是探索者的所有可行的 a 啦。

如果在某个地点 s1, 探索者计算了他能有的两个行为, a1/a2=left/right, 计算结果是 Q(s1, a1) > Q(s1, a2), 那么探索者就会选择 left 这个行为. 这就是 Q learning 的行为选择简单规则。

In [10]:
import numpy as np
import pandas as pd
import time

np.random.seed(2)  # 可重复

N_STATES = 6   # 1维世界的宽度
ACTIONS = ['left', 'right']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度（exploitation rate）
ALPHA = 0.1     # 学习率（exploration rate）
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 13   # maximum episodes
FRESH_TIME = 0.3    # fresh time for one move

对于 tabular Q learning, 我们必须将所有的 Q values (行为值) 放在 q_table 中, 更新 q_table 也是在更新他的行为准则. q_table 的 index 是所有对应的 state (探索者位置), columns 是对应的 action (探索者行为)。

In [8]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table 全0初始
        columns=actions,    # columns 对应的是行为名称
    )
    # print(table)    # show table
    return table

In [ ]:
"""
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000
"""

接着定义探索者是如何挑选行为的. 这是我们引入 <span class="mark">epsilon greedy</span> 的概念. 因为在初始阶段, 随机的探索环境, 往往比固定的行为模式要好, 所以这也是累积经验的阶段, 我们希望探索者不会那么贪婪(greedy). 所以 EPSILON 就是用来控制贪婪程度的值. EPSILON 可以随着探索时间不断提升(越来越贪婪), 不过在这个例子中, 我们就固定成 EPSILON = 0.9, 90% 的时间是选择最优策略, 10% 的时间来探索。

In [1]:
# 在某个state地点，选择行为
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :] # 选出这个state的所有action值
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # 非贪婪 or 或者这个 state 还没有探索过
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        action_name = state_actions.idxmax()    # 贪婪模式
    return action_name

做出行为后, 环境也要给我们的行为一个反馈, 反馈出下个 state (S_) 和 在上个 state (S) 做出 action (A) 所得到的 reward (R). 这里定义的规则就是, <span class="mark">只有当 o 移动到了 T, 探索者才会得到唯一的一个奖励, 奖励值 R=1, 其他情况都没有奖励。</span>

In [3]:
def get_env_feedback(S, A):
    if A == 'right':    # 右移
        if S == N_STATES - 2:   # 找到宝藏了，停止
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # 左移
        R = 0
        if S == 0:
            S_ = S  # 撞墙了
        else:
            S_ = S - 1
    return S_, R

In [4]:
def update_env(S, episode, step_counter):
    # 环境更新
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [ ]:
"""
-o---T
# T 就是宝藏的位置, o 是探索者的位置
"""

### 主循环

随机初始化 Q(s,a) Table  
循环调用（episode）：  
>初始化 state s  
>循环调用（episode中的每一步）：  
>>从 Q Table 中，根据 s，选择 action a （使用 epsilon 贪婪法）  
>>采取 a，观察下一个state s'以及奖励 r  
>>更新 Q table，$Q(s,a) \leftarrow Q(s,a)+\alpha[r+\gamma \max Q(s',a')-Q(s,a)]$  
>>$s \leftarrow s'$  
>直到 s 停止

In [6]:
def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES): # 回合
        step_counter = 0
        S = 0 # 回合初始位置
        is_terminated = False # 回合是否结束
        update_env(S, episode, step_counter) # 环境更新
        while not is_terminated:
            A = choose_action(S, q_table) # 选择行为
            S_, R = get_env_feedback(S, A)  # 实施行为，并得到环境的反馈
            q_predict = q_table.loc[S, A] # 估算的（状态-行为）值
            if S_ != 'terminal': # 如果还没有找到宝藏
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # 实际的（状态-行为）值
            else:
                q_target = R     # 实际的（状态-行为）值
                is_terminated = True

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # 更新 Q Table
            S = S_  # 探索者移动到下一个 state

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table

In [11]:
if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

                                
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000
